### Load Dataset

In [1]:
import pandas as pd
import numpy as np

## Takes a few seconds to load
file_path = '../data/raw/Orders_Master_Data(in).xlsx'
raw_data = pd.read_excel(file_path) 

### Dataset Cleaning

In [2]:
df = raw_data.copy()
df.columns = df.columns.str.replace(' ', '_')
df.columns = df.columns.str.lower()
df = df.drop_duplicates()

df['date'] = pd.to_datetime(df['date'], format='%d.%m.%Y')
df.set_index('date', inplace=True)
df.rename(columns={"median_ticket_(‚ç¨)": "median_ticket",
                   "prom_contacts_month":"promotor_visits",
                   "tel_contacts_month":"promotor_calls"},
                   inplace=True)

df['city'] =  df['city'].apply(lambda x: 'Cadiz' if x=='C√°diz' else x)
df['city'] =  df['city'].apply(lambda x: 'Castellon' if x=='Castell√≥n' else x)
df['city'] =  df['city'].apply(lambda x: 'Cordoba' if x=='C√≥rdoba' else x)

# Time Features
df['day_of_week'] = df.index.dayofweek
df['month'] = df.index.month
df['week'] = df.index.isocalendar().week

# Cost Calculation
logistics_cost = 10 #EUR per order
visit_cost = 15 #EUR per visit

df['order_normalized'] = np.where(df['number_of_orders'] > 0, 1, 0) 
df['cost'] = df['order_normalized'] * logistics_cost + df['promotor_visits'] * visit_cost
df['profit'] = df['income'] - df['cost']

### Final Orders df

In [3]:
# Reorganize columns in a logical order
column_order = [
    # Client & Geographic Information
    'client_id',
    'city', 
    'channel',
    'promotor_id',
    
    # Core Business Metrics
    'number_of_orders',
    'volume',
    'income',
    'median_ticket',
    
    # Contact & Visit Metrics
    'promotor_visits',
    'promotor_calls',
    
    # Calculated Financial Metrics
    'order_normalized',
    'cost',
    'profit',

    # Time Features
    'month', 
    'week',
    'day_of_week',
]

# Reorder the DataFrame columns
df = df[column_order]


### Filtered Orders df

In [20]:
#No Volume and no Income
# filter=(df['volume']==0) & (df['income']==0) & (df['number_of_orders']==0)
# filter=(df['volume']==0)
filter=(df['volume']==0) & (df['income']==0)

filtered_df = df[~filter]

# Print the difference in shapes between df and filtered_df
print("Original df shape:", df.shape)
print("Filtered df shape:", filtered_df.shape)
print("Difference in rows:", df.shape[0] - filtered_df.shape[0])
print("Rows removed (volume=0 AND income=0):", df.shape[0] - filtered_df.shape[0])
print("Percentage of rows removed:", f"{((df.shape[0] - filtered_df.shape[0]) / df.shape[0] * 100):.2f}%")

Original df shape: (1014965, 16)
Filtered df shape: (1001824, 16)
Difference in rows: 13141
Rows removed (volume=0 AND income=0): 13141
Percentage of rows removed: 1.29%


### Monthly Clients df

In [5]:
aggregation_rules = {
    'city':               lambda x: x.mode()[0] if not x.mode().empty else None, 
    'channel':            lambda x: x.mode()[0] if not x.mode().empty else None, 
    'promotor_id':        lambda x: x.mode()[0] if not x.mode().empty else None,  
    'order_normalized':   'sum',
    'volume':             'sum',    
    'income':             'sum',
    'cost':               'sum',
    'profit':             'sum',
    'median_ticket':      ['median','min','max','std'],
    'promotor_visits':    'median',
    'promotor_calls':     'median'
}

clients_monthly = filtered_df.groupby(['client_id', 'month']).agg(aggregation_rules)

# Flatten column MultiIndex into clean names
new_cols = []
for orig_col, agg_func in clients_monthly.columns:
    if agg_func in ('first', '<lambda>'):
        # keep the original name for single-function aggs
        new_cols.append(orig_col)
    else:
        # join field + function for multi-aggs
        new_cols.append(f"{orig_col}_{agg_func}")
clients_monthly.columns = new_cols

clients_monthly.rename(columns={
    'order_normalized_sum':    'total_orders',
    'volume_sum':              'total_volume',
    'income_sum':              'total_income',
    'cost_sum':                'total_cost',
    'profit_sum':              'total_profit',
    'median_ticket_median':    'median_ticket',
    'median_ticket_min':       'median_ticket_min',
    'median_ticket_max':       'median_ticket_max',
    'median_ticket_std':       'median_ticket_std',
    'promotor_visits_median':  'median_promotor_visits',
    'promotor_calls_median':   'median_promotor_calls',
}, inplace=True)


### Monthly Efficiency

**Note: Given that Max efficiency is 20, the efficiency is filled in as 20 when promotor visits is equal to zero.**

In [78]:
# efficiency = clients_monthly[['total_orders','median_promotor_visits']].copy()
# efficiency['efficiency'] = (efficiency['total_orders'] / efficiency['median_promotor_visits'].replace(0, np.nan)).fillna(0)

# cap_value = efficiency['efficiency'].max()
# efficiency.loc[efficiency['median_promotor_visits'] == 0, 'efficiency'] = cap_value

# efficiency.rename(columns={'total_orders': 'frequency'}, inplace=True)
# efficiency.drop(columns=['median_promotor_visits'], inplace=True)
# efficiency_monthly = efficiency.copy()

# efficiency = efficiency.groupby('client_id').agg({'frequency': ['median','min','max','std'],
#                                                   'efficiency': ['median','min','max','std']})

# # Flatten column MultiIndex into clean names
# new_cols = []
# for orig_col, agg_func in efficiency.columns:
#     if agg_func in ('first', '<lambda>'):
#         # keep the original name for single-function aggs
#         new_cols.append(orig_col)
#     else:
#         # join field + function for multi-aggs
#         new_cols.append(f"{orig_col}_{agg_func}")
# efficiency.columns = new_cols

# efficiency

In [66]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

efficiency_monthly = clients_monthly[['total_orders','median_promotor_visits','median_promotor_calls']].copy()
efficiency_monthly['efficiency'] = efficiency_monthly['total_orders'] / efficiency_monthly['median_promotor_visits']

efficiency_monthly['efficiency'] = efficiency_monthly['efficiency'].replace([np.inf, -np.inf], np.inf)

# Step 1: Separate finite and infinite values
finite_mask = np.isfinite(efficiency_monthly['efficiency'])
infinite_mask = ~finite_mask

# Step 2: Scale only finite values
scaler = MinMaxScaler()
efficiency_monthly.loc[finite_mask, 'efficiency_scaled'] = scaler.fit_transform(efficiency_monthly.loc[finite_mask, ['efficiency']])

# Step 3: Assign a fixed high score to infs (e.g., 1.5, or max(finite) + margin)
inf_value = efficiency_monthly['efficiency_scaled'].max() + 0.5
efficiency_monthly.loc[infinite_mask, 'efficiency_scaled'] = inf_value

efficiency_monthly.rename(columns={'total_orders': 'frequency'}, inplace=True)

efficiency_monthly

frequency  median_promotor_visits  median_promotor_calls  \
client_id month                                                             
100006690 1              2                     2.0                    0.0   
          2              2                     2.0                    0.0   
          3              2                     2.0                    0.0   
          4              3                     2.0                    0.0   
          5              3                     2.0                    0.0   
...                    ...                     ...                    ...   
999976985 7              3                     1.0                    0.0   
          8              4                     1.0                    0.0   
          9              1                     1.0                    0.0   
          10             3                     1.0                    0.0   
          11             2                     1.0                    0.0   

                 efficiency  efficiency_scaled  
client_id month                                 
100006690 1             1.0              0.050  
          2             1.0              0.050  
          3             1.0              0.050  
          4             1.5              0.075  
          5             1.5              0.075  
...                     ...                ...  
999976985 7             3.0              0.150  
          8             4.0              0.200  
          9             1.0              0.050  
          10            3.0              0.150  
          11            2.0              0.100  

[360413 rows x 5 columns]

### Year Efficiency df

In [68]:
efficiency = efficiency_monthly.groupby('client_id').agg({'frequency': 'median',
                                                          'median_promotor_visits': 'sum',
                                                          'median_promotor_calls': 'sum',
                                                          'efficiency': 'median',
                                                          'efficiency_scaled': 'median'})

efficiency.rename(columns={'median_promotor_visits': 'total_promotor_visits',
                            'median_promotor_calls': 'total_promotor_calls'}, inplace=True)

efficiency

,frequency,total_promotor_visits,total_promotor_calls,efficiency,efficiency_scaled
client_id,,,,,
100006690,2.0,24.0,0.0,1.00,0.0500
100008050,1.0,0.0,20.0,inf,1.5000
100042162,1.0,44.0,0.0,0.25,0.0125
100046227,4.0,8.0,8.0,2.00,0.1000
100125158,3.0,10.0,0.0,3.00,0.1500
...,...,...,...,...,...
999934164,2.0,9.0,27.0,2.00,0.1000
999940211,1.0,0.0,0.0,inf,1.5000
999940578,1.0,22.0,0.0,0.50,0.0250


### Clients df

In [77]:
aggregation_rules = {
    'city':               lambda x: x.mode()[0] if not x.mode().empty else None, 
    'channel':            lambda x: x.mode()[0] if not x.mode().empty else None, 
    'promotor_id':        lambda x: x.mode()[0] if not x.mode().empty else None,  
    'order_normalized':   'sum',
    'volume':             'sum',    
    'income':             'sum',
    'cost':               'sum',
    'profit':             'sum',
    # 'median_ticket':      ['median','min','max','std'],
    'median_ticket':      'median',
    'promotor_visits':    'median',
    'promotor_calls':     'median',
}
clients = filtered_df.groupby('client_id').agg(aggregation_rules)

# # Flatten column MultiIndex into clean names
# new_cols = []
# for orig_col, agg_func in clients.columns:
#     if agg_func in ('first', '<lambda>'):
#         # keep the original name for single-function aggs
#         new_cols.append(orig_col)
#     else:
#         # join field + function for multi-aggs
#         new_cols.append(f"{orig_col}_{agg_func}")
# clients.columns = new_cols

# clients = clients.merge(efficiency,on='client_id',how='left')

clients.rename(columns={
    # 'city_first': 'city',
    # 'channel_first': 'channel',
    # 'promotor_id_first': 'promotor_id',
    # 'frequency_median': 'frequency',
    # 'efficiency_median': 'efficiency',
    'order_normalized':    'total_orders',
    'volume':              'total_volume',
    'income':              'total_income',
    'cost':                'total_cost',
    'profit':              'total_profit',
    'median_ticket':    'median_ticket',
#     'median_ticket_min':       'median_ticket_min',
#     'median_ticket_max':       'median_ticket_max',
#     'median_ticket_std':       'median_ticket_std',
    'promotor_visits':  'median_monthly_promotor_visits',
    'promotor_calls':   'median_monthly_promotor_calls',
#     'frequency_min': 'frequency_min',
#     'frequency_max': 'frequency_max',
#     'frequency_std': 'frequency_std',
#     'efficiency_min': 'efficiency_min',
#     'efficiency_max': 'efficiency_max',
#     'efficiency_std': 'efficiency_std',
}, inplace=True)

# cols = [
#     'city',
#     'channel',
#     'promotor_id',
#     'frequency',
#     'efficiency',
#     'total_orders',
#     'total_volume',
#     'total_income',
#     'total_cost',
#     'total_profit',
#     'median_ticket',
#     'median_ticket_min',
#     'median_ticket_max',
#     'median_ticket_std',
#     'median_promotor_visits',
#     'median_promotor_calls',
#     # 'frequency_min',
#     # 'frequency_max',
#     # 'frequency_std',
#     # 'efficiency_min',
#     # 'efficiency_max',
#     # 'efficiency_std',
# ]
# clients = clients[cols]

# low_ticket_threshold = 80

# #Defining a function to pick the quadrant label
# def assign_quadrant(row):
#     high_ticket   = row['median_ticket'] >  low_ticket_threshold
#     efficient    = row['efficiency'] > 1
#     if   high_ticket and efficient:    return 'HighTicket_Efficient'
#     elif not high_ticket and efficient: return 'LowTicket_Efficient'
#     elif high_ticket and not efficient: return 'HighTicket_Inefficient'
#     else:                              return 'LowTicket_Inefficient'

# clients['class'] = clients.apply(assign_quadrant, axis=1)
# #Compute per-month averages and gap
# clients['zero_visit_flag'] = clients['median_promotor_visits'] == 0
# clients['avg_orders_per_month'] = clients['total_orders'] / 12
# clients['avg_visits_per_month'] = clients['median_promotor_visits'] / 12
# clients['visit_order_gap'] = clients['avg_visits_per_month'] - clients['avg_orders_per_month']

# #Direct inefficiency cost
# clients['inefficiency_cost'] = clients['visit_order_gap'] * 15

# #Profit per visit & opportunity cost
# clients['profit_per_visit'] = clients.apply(
#     lambda r: r['total_profit'] / r['median_promotor_visits'] if r['median_promotor_visits'] > 0 else 0,
#     axis=1
# )
# clients['opportunity_cost'] = clients['visit_order_gap'] * clients['profit_per_visit']

clients


,city,channel,promotor_id,total_orders,total_volume,total_income,total_cost,total_profit,median_ticket,median_monthly_promotor_visits,median_monthly_promotor_calls
client_id,,,,,,,,,,,
100006690,Madrid,AR,275609911,22,1658.706,1494.53,880,614.53,60.990,2.0,0.0
100008050,Barcelona,AR,368568690,14,3982.000,1905.59,140,1765.59,132.370,0.0,2.0
100042162,Barcelona,HR,455263770,14,1812.850,2243.30,980,1263.30,128.125,4.0,0.0
100046227,Barcelona,AR,454554895,16,4590.180,2273.12,640,1633.12,132.640,2.0,2.0
100125158,Cadiz,HR,483340469,25,1266.500,2204.24,625,1579.24,85.810,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
999934164,Barcelona,HR,480416490,23,691.000,785.30,575,210.30,38.330,1.0,3.0
999940211,Barcelona,AR,908993212,3,557.820,260.55,30,230.55,76.430,0.0,0.0
999940578,Madrid,AR,275609911,13,1101.524,1044.61,520,524.61,68.260,2.0,0.0


### Export Final dfs

In [8]:
import os

# Create processed data directory if it doesn't exist
processed_dir = '../data/processed'
if not os.path.exists(processed_dir):
    os.makedirs(processed_dir)
    print(f"Created directory: {processed_dir}")

df.to_csv('../data/processed/orders_raw.csv', index=True)
filtered_df.to_csv('../data/processed/orders.csv', index=True)
clients.to_csv('../data/processed/clients.csv', index=True)
clients_monthly.to_csv('../data/processed/clients_monthly.csv', index=True)

In [9]:
clients

,city,channel,promotor_id,frequency,efficiency,total_orders,total_volume,total_income,total_cost,total_profit,...,efficiency_max,efficiency_std,class,zero_visit_flag,avg_orders_per_month,avg_visits_per_month,visit_order_gap,inefficiency_cost,profit_per_visit,opportunity_cost
client_id,,,,,,,,,,,,,,,,,,,,,
100006690,Madrid,AR,275609911,2.0,1.00,22,1658.706,1494.53,880,614.53,...,1.5,0.417424,LowTicket_Inefficient,False,1.833333,0.166667,-1.666667,-25.00,307.265,-512.108333
100008050,Barcelona,AR,368568690,1.0,20.00,14,3982.000,1905.59,140,1765.59,...,20.0,0.000000,HighTicket_Efficient,True,1.166667,0.000000,-1.166667,-17.50,0.000,-0.000000
100042162,Barcelona,HR,455263770,1.0,0.25,14,1812.850,2243.30,980,1263.30,...,0.5,0.116775,HighTicket_Inefficient,False,1.166667,0.333333,-0.833333,-12.50,315.825,-263.187500
100046227,Barcelona,AR,454554895,4.0,2.00,16,4590.180,2273.12,640,1633.12,...,2.0,0.000000,HighTicket_Efficient,False,1.333333,0.166667,-1.166667,-17.50,816.560,-952.653333
100125158,Cadiz,HR,483340469,3.0,3.00,25,1266.500,2204.24,625,1579.24,...,4.0,0.971825,HighTicket_Efficient,False,2.083333,0.083333,-2.000000,-30.00,1579.240,-3158.480000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999934164,Barcelona,HR,480416490,2.0,2.00,23,691.000,785.30,575,210.30,...,4.0,1.236033,LowTicket_Efficient,False,1.916667,0.083333,-1.833333,-27.50,210.300,-385.550000
999940211,Barcelona,AR,908993212,1.0,20.00,3,557.820,260.55,30,230.55,...,20.0,0.000000,LowTicket_Efficient,True,0.250000,0.000000,-0.250000,-3.75,0.000,-0.000000
999940578,Madrid,AR,275609911,1.0,0.50,13,1101.524,1044.61,520,524.61,...,1.0,0.202260,LowTicket_Inefficient,False,1.083333,0.166667,-0.916667,-13.75,262.305,-240.446250


In [10]:
efficiency

,frequency_median,frequency_min,frequency_max,frequency_std,efficiency_median,efficiency_min,efficiency_max,efficiency_std
client_id,,,,,,,,
100006690,2.0,1,3,0.834847,1.00,0.50,1.5,0.417424
100008050,1.0,1,2,0.516398,20.00,20.00,20.0,0.000000
100042162,1.0,1,2,0.467099,0.25,0.25,0.5,0.116775
100046227,4.0,4,4,0.000000,2.00,2.00,2.0,0.000000
100125158,3.0,1,4,0.971825,3.00,1.00,4.0,0.971825
...,...,...,...,...,...,...,...,...
999934164,2.0,1,4,1.236033,2.00,1.00,4.0,1.236033
999940211,1.0,1,1,0.000000,20.00,20.00,20.0,0.000000
999940578,1.0,1,2,0.404520,0.50,0.50,1.0,0.202260
